# Localização - Alocação: O Problema de maximizar a cobertura

Autor: Gustavo Eduardo Marcatti  
20 de março de 2021

Bibliotecas necessárias

In [1]:
import pandas as pd # Manipulação de dados
import geopandas as gpd # Manipulação de dados espaciais

## 1. Importar dados

Arquivos das feições espaciais, disponibilizadas nos links a seguir, no formato geojson (modelo de transmissão de dados espaciais no formato texto, derivado do formato geral json, muito popular na internet).

In [2]:
arq_antenas = "https://raw.githubusercontent.com/gmarcatti/prog-python/main/dados/antenas.geojson"
arq_infra = "https://raw.githubusercontent.com/gmarcatti/prog-python/main/dados/infraestrutura_projeto.geojson"

Importar as feições espaciais e definir os campos `cod_antena` e `Cod_proj` como índices das respectivas tabelas

In [3]:
antenas = gpd.read_file(arq_antenas)
antenas = antenas.set_index('cod_antena')
infra = gpd.read_file(arq_infra)
infra = infra.set_index('Cod_proj')

## 2. Pre-processamento

Computar matriz de distâncias: matriz com i linhas (pontos de infraestruturas) e j colunas (antenas)

In [4]:
dist_matriz = infra.geometry.apply(lambda g: antenas.distance(g))
dist_matriz

<ipython-input-4-bbddec8a3aa7>:1: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  dist_matriz = infra.geometry.apply(lambda g: antenas.distance(g))


cod_antena,a1,a2,a3,a4,a5,a6,a7,a8,a9
Cod_proj,,,,,,,,,
30,182717.325777,90736.935322,13956.693660,90411.290832,67867.349495,75798.424238,35909.029358,264971.770873,148224.852598
40,151750.541574,160798.513512,82930.415350,100944.305358,115575.513862,34924.440998,37312.825792,220987.285865,114695.205596
150,71991.908453,222842.725681,175139.917888,87227.245537,130292.700800,140212.876696,183270.104091,146398.638567,83197.297754
5350,204837.721622,46870.902514,41216.646488,99663.530660,49170.032219,119890.062667,90951.603153,292067.835877,175456.192506
210,76480.991643,210155.899604,133950.674098,86135.949454,132558.922485,51805.526392,107350.686860,143074.440035,41563.416855
...,...,...,...,...,...,...,...,...,...
7050,164987.533416,118815.763186,40589.296620,85341.940346,80931.702403,50658.592242,14880.801969,243759.503784,128956.998196
7130,118234.208850,147376.343114,71726.783210,49756.003671,76220.781518,22982.433389,57770.738773,199172.496024,82776.819260
7140,150564.023546,102283.404711,59089.926744,44667.660640,10116.523683,90121.533728,91612.137354,238690.367938,124403.418857


Considerar apenas os pontos da matriz em que a distância é menor do que o alcance da respectiva antena

In [5]:
cond_dist = dist_matriz < antenas['alcance_m']
dist_alcance = dist_matriz[cond_dist]
dist_alcance

cod_antena,a1,a2,a3,a4,a5,a6,a7,a8,a9
Cod_proj,,,,,,,,,
30,NaN,NaN,13956.693660,NaN,NaN,NaN,NaN,NaN,NaN
40,NaN,NaN,NaN,NaN,NaN,34924.440998,NaN,NaN,NaN
150,71991.908453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5350,NaN,46870.902514,41216.646488,NaN,49170.032219,NaN,NaN,NaN,NaN
210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41563.416855
...,...,...,...,...,...,...,...,...,...
7050,NaN,NaN,40589.296620,NaN,NaN,NaN,14880.801969,NaN,NaN
7130,NaN,NaN,NaN,49756.003671,NaN,22982.433389,NaN,NaN,NaN
7140,NaN,NaN,NaN,44667.660640,10116.523683,NaN,NaN,NaN,NaN


Converter a matriz formato padrão com i pontos de estruturas e j antenas (i *x* j) em uma matriz com i * j linhas e 3 colunas (pontos de estrutura, antenas e distâncias).

In [6]:
dist_alcance = dist_alcance.stack().reset_index()
dist_alcance

,Cod_proj,cod_antena,0
0,30,a3,13956.693660
1,40,a6,34924.440998
2,150,a1,71991.908453
3,5350,a2,46870.902514
4,5350,a3,41216.646488
...,...,...,...
218,7130,a4,49756.003671
219,7130,a6,22982.433389
220,7140,a4,44667.660640
221,7140,a5,10116.523683


Contabilização dos pontos de infraestruturas alcançados por cada uma das 9 antenas 

In [7]:
dist_alcance.groupby('cod_antena')[0].count()

cod_antena
a1    44
a2     9
a3    32
a4    37
a5    25
a6    22
a7    10
a8     4
a9    40
Name: 0, dtype: int64

## 3. Solução aproximada utilizando procedimento guloso

Preparação dos dados para execução do algoritmo. Dicionário (dict) em que a chave (key) é o código das antenas e os valores (value) é um conjunto (set) com os pontos de infraestruturas alcançados por cada uma das antenas

In [8]:
antena_uni = dist_alcance['cod_antena'].unique()
antena_infra = {}
for ant in antena_uni:
    infra_i = set(dist_alcance[dist_alcance['cod_antena'] == ant]['Cod_proj'])
    antena_infra[ant] = infra_i

antena_infra

{'a3': {30,
  250,
  370,
  1170,
  1210,
  1290,
  1330,
  2200,
  2270,
  2352,
  2420,
  2595,
  3550,
  3940,
  3950,
  4090,
  4210,
  4880,
  5020,
  5350,
  5400,
  5490,
  5740,
  5790,
  5890,
  6010,
  6140,
  6255,
  6400,
  6630,
  6760,
  7050},
 'a6': {40,
  250,
  870,
  1020,
  1630,
  1670,
  2170,
  2190,
  2330,
  2820,
  4830,
  4880,
  5080,
  5210,
  5230,
  5310,
  6150,
  6380,
  6570,
  6600,
  6850,
  7130},
 'a1': {150,
  330,
  440,
  460,
  610,
  680,
  690,
  1590,
  1620,
  1960,
  2130,
  2290,
  2500,
  2840,
  2850,
  3260,
  3670,
  3840,
  3860,
  3980,
  4020,
  4080,
  4160,
  4540,
  4570,
  4660,
  4940,
  4950,
  5010,
  5130,
  5540,
  5580,
  5590,
  5620,
  5860,
  5940,
  6000,
  6070,
  6290,
  6560,
  6730,
  6750,
  6790,
  6900},
 'a2': {1210, 1600, 2352, 3770, 3940, 3950, 5350, 5890, 6760},
 'a5': {310,
  370,
  550,
  1010,
  1170,
  1210,
  1330,
  2200,
  2400,
  2420,
  2490,
  2530,
  2595,
  3950,
  4210,
  4390,
  4820,
  4900,


Procedimento de maximização da cobertura propriamente dito

In [9]:
n_antena = 9 # quantidade de antenas a serem selecionadas
infra_nao_coberto = set(dist_alcance['Cod_proj'])
antena_final = set()
while infra_nao_coberto:
    melhor_antena = None
    infra_coberta = set()
    for ant, inf in antena_infra.items():
        cobertos = infra_nao_coberto & inf
        if len(cobertos) > len(infra_coberta):
            melhor_antena = ant
            infra_coberta = cobertos  
    infra_nao_coberto -= infra_coberta
    antena_final.add(melhor_antena)
    print("Antena:", melhor_antena, "; Contribuição:", len(infra_coberta))
    if len(antena_final) >= n_antena: break


Antena: a1 ; Contribuição: 44
Antena: a3 ; Contribuição: 32
Antena: a4 ; Contribuição: 27
Antena: a6 ; Contribuição: 12
Antena: a5 ; Contribuição: 3
Antena: a2 ; Contribuição: 2
Antena: a7 ; Contribuição: 2
Antena: a9 ; Contribuição: 1
Antena: a8 ; Contribuição: 1
